In [1]:
import pandas as pd
import scipy.sparse as sp
import numpy as np
urmPath = "../Input/interactions_and_impressions.csv"
icmLenghtPath = "../Input/data_ICM_length.csv"

In [2]:
dataset = pd.read_csv(urmPath)
icm = pd.read_csv(icmLenghtPath)
dataset = dataset.drop(columns=['Impressions'])
datasetGroup = dataset.groupby(['UserID', 'ItemID', 'Data']).size()

C:\Users\Mike\AppData\Local\Temp\ipykernel_27696\1584756250.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(urmPath)


In [15]:
datasetGroup.head(200)

UserID  ItemID  Data
0       11      1        1
        21      0        3
                1        9
        22      1       15
        24      1        1
                        ..
2       17396   0        1
        17970   0        1
        18001   0        1
        18484   1        3
        19057   0        1
Length: 200, dtype: int64

In [4]:
icmOO = sp.coo_matrix((icm["data"].values, (icm["item_id"].values, icm["feature_id"].values)))

In [12]:
datasetCOO = sp.coo_matrix((dataset["Data"].values, (dataset["UserID"].values, dataset["ItemID"].values)))
userIDS = dataset['UserID'].unique()
itemIDS = dataset['ItemID'].unique()

ICM = np.zeros((27968, 1), dtype=int)
for item in range(len(icmOO.data)):
    ICM[icmOO.row[item]] = icmOO.data[item]

list_user = []

URM = np.zeros((len(userIDS), len(itemIDS)), dtype=float)

for x in range(len(datasetCOO.data)):
    if datasetCOO.row[x] not in list_user and datasetCOO.data[x] == 0:
        if datasetGroup[datasetCOO.row[x]][datasetCOO.col[x]][0] >= ICM[datasetCOO.col[x]]:
            URM[datasetCOO.row[x]][datasetCOO.col[x]] = 10
            list_user.append(datasetCOO.row[x])
        elif datasetCOO.row[x] not in list_user:
            URM[datasetCOO.row[x]][datasetCOO.col[x]] = 5 + (datasetGroup[datasetCOO.row[x]][datasetCOO.col[x]][0] / ICM[datasetCOO.col[x]] * 10)
            list_user.append(datasetCOO.row[x])
    elif datasetCOO.data[x] == 1 and URM[datasetCOO.row[x]][datasetCOO.col[x]] < 4:
        URM[datasetCOO.row[x]][datasetCOO.col[x]] = URM[datasetCOO.row[x]][datasetCOO.col[x]] + 1

URM

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
DF = pd.DataFrame(URM)
DF.to_csv('../Input/URM.csv')